# Reconstruction of DOSY NMR signals - Part I

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import random

## 1) Generation of synthetic data


$\underline{\textbf{Q1.}}$ Let's read the signal data from the file "signal.dat" 

In [ ]:
N = 200
signal = np.zeros(N)

filepath = 'signal.txt'  
with open(filepath) as fp:  
    line = fp.readline()
    cnt = 1
    while line:
        signal[cnt-1] = np.float64(line)
        line = fp.readline()
        cnt += 1
    
signal

$\underline{\textbf{Q2.}}$  We create first T using an exponential sampling strategy:

In [ ]:
T_min = 1 
T_max = 1000
T = np.array([T_min*np.exp(-n*(np.log(T_min/T_max))/(N-1)) for n in range(N)],dtype=np.float64)
T

$\underline{\textbf{Q3.}}$ Display the original signal $\bar{x}$ as a function of T

In [ ]:
plt.plot(np.log(T),signal)

$\underline{\textbf{Q4.}}$  Create t using a regular sampling strategy 

In [ ]:
M = 50 
t_min = 0
t_max = 1.5
t = np.array([t_min + ((m-1)/(M-1))*(t_max - t_min) for m in range(1,M+1)],dtype=np.float64)
t

 $\underline{\textbf{Q4.}}$ Construct matrix K using (3)

In [ ]:
##### I should change this (no loop) ###########

K = np.zeros((M,N))

for n in range(N):
    for m in range(M):
        K[m,n] = np.exp(- t[m]*T[n])

rank $K = \min(n,m)$ so $K^\top K$ is symetric positive.

$\underline{\textbf{Q5.}}$ Simulate the noisy data according to model (2), by taking $w \sim \mathcal{N} (0, \sigma^2 I_{M})$ with $\sigma = 0.01z^{(1)}$ , where $z = K\bar{x}$

In [ ]:
x = signal
xmin = min(x)
xmax = max(x)
z = np.dot(K,x)
sigma = 0.01*z[0]
mean = np.zeros(M)
cov  = (sigma**2)*np.eye(M)

w = np.random.multivariate_normal(mean, cov)
y = z + w

$\underline{\textbf{Q6.}}$ Display the resulting noisy data y as a function of t

In [ ]:
plt.plot(t,y)

## 2) Comparison of regularization strategies

###  Smoothness prior

Defintion of D

In [ ]:
D = np.eye(N)

for n in range(N-1):
    D[n+1,n] = -1

D[0,N-1] = -1

We consider the problem:

$\inf_{x\in\mathbb{R}^N}\ \frac{1}{2}\ \lVert Kx - y \rVert^2_2 + \frac{\beta}{2}\ \lVert Dx \rVert^2_2 = \inf_{x\in\mathbb{R}^N}\ J(x)$

J is coercive so a minimum exists. J is striclty convex because $\nabla^2 J = K^\top K + \beta\ D^\top D$ is symetric positive. Therefore the minimum is unique.

$\underline{\textbf{Q3.}}$ Solving the optimization problem. The optimality condition gives us the exact solution.

In [ ]:
beta = 1
x_hat = np.dot(np.linalg.inv(np.dot(K.T,K) + beta*np.dot(D.T,D)),np.dot(K.T,y))

In [ ]:
def normalized_MSE(x,y):
    return( (np.linalg.norm(x-y)/np.linalg.norm(y))**2)

In [ ]:
print(normalized_MSE(x_hat,x))
plt.plot(np.log(T),x_hat)
plt.plot(np.log(T),x)
plt.grid()

$\underline{\textbf{Q4.}}$ Tunning $\beta$ with a grid search 

In [ ]:
beta_grid = np.exp(np.log(10)*np.linspace(-2,2,5))
errors = np.zeros(beta_grid.shape[0])

for i in range(beta_grid.shape[0]):
    beta = beta_grid[i]
    x_hat = np.dot(np.linalg.inv(np.dot(K.T,K) + beta*np.dot(D.T,D)),np.dot(K.T,y))
    errors[i] = normalized_MSE(x_hat,x)
    print("MSE : {}, beta : {}".format(normalized_MSE(x_hat,x),beta))
    plt.plot(np.log(T),x_hat)
    plt.plot(np.log(T),x)
    plt.grid()
    plt.show()

### Smoothness prior + constraints 


We consider the problem:

$\inf_{x\in\mathbb{R}^N}\ \frac{1}{2}\ \lVert Kx - y \rVert^2_2 + \frac{\beta}{2}\ \lVert Dx \rVert^2_2 + \iota_{[x_{min},x_{max}]^N}(x)= \inf_{x\in[x_{min},x_{max}]^N}\ J(x)$

$[x_{min},x_{max}]^N$ is compact and L is continuous so L admits a minimum on $[x_{min},x_{max}]^N$. L is striclty convex because $\nabla^2 J = K^\top K + \beta\ D^\top D$ is symetric positive. Thus, the minimum is unique.

We consider a forward-backward algorithm.

In [ ]:
beta_grid = np.exp(np.log(10)*np.linspace(-3,1,5))

def proximal_gradient_smooth_cons(lamb,K,y,T,xmin,xmax,display = False):
    tol = 1e-8
    m = K.shape[0]
    n = K.shape[1]
    u = np.zeros((n,1))
    w,v = np.linalg.eig(np.dot(K.T,K) + lamb * np.dot(D.T,D))
    L = max(w)
    epsilon = 1/L
    norm2 = np.linalg.norm(np.dot(K,u) - y.reshape(m,1))
    normD = np.linalg.norm(np.dot(D,u))
    obj = 0.5 * norm2 **2 + 0.5 * lamb * normD ** 2 + ind(u,xmin,xmax)
    last_obj = 0
    if display:
        print("Objectif : {}, ||Ku-y||^2 : {}, ||Du||^2 : {}, ind : {}".format(obj,norm2**2,normD**2,ind(u,xmin,xmax)))
    
    while abs(obj - last_obj) > tol:
        w = u - epsilon * (np.dot(K.T,np.dot(K,u) - y.reshape(m,1)) + lamb * np.dot(D.T,np.dot(D,u)))
        last_obj = obj
        u = prox_cons(w,xmin,xmax) 
        norm2 = np.linalg.norm(np.dot(K,u) - y.reshape(m,1))
        normD = np.linalg.norm(np.dot(D,u))
        obj = 0.5 * norm2 **2 + 0.5*lamb * normD ** 2 + ind(u,xmin,xmax)
        if display:
            print("Objectif : {}, ||Ku-y||^2 : {}, ||Du||^2 : {}, ind : {}".format(obj,norm2**2,normD**2,ind(u,xmin,xmax)))
        
    return u
        
def prox_cons(w,xmin,xmax):
    m = w.shape[0]
    out = np.zeros((m,1))
    for index in range(m):
        value = w[index,0]
        if value >  xmax:
            out[index,0] = xmax
        elif value <  xmin:
            out[index,0] = xmin
        elif value >= xmin and value <= xmax:
            out[index,0] = value
    return out

def ind(x,xmin,xmax):
    n = x.shape[0]
    for i in range(n):
        if x[i] > xmax or x[i] < xmin:
            return 1e30
    return 0

for i in range(beta_grid.shape[0]):
    beta = beta_grid[i]
    x_hat = proximal_gradient_smooth_cons(beta,K,y,T,xmin,xmax) #,True
    print("MSE : {}, beta : {}\n".format(normalized_MSE(x_hat,x.reshape(200,1)),beta))
    plt.plot(np.log(T),x_hat)
    plt.plot(np.log(T),x)
    plt.grid()
    plt.show()

### Sparsity 


We consider the problem:

$\inf_{x\in\mathbb{R}^N}\ \frac{1}{2}\ \lVert Kx - y \rVert^2_2 + \beta\ \lVert x \rVert_1 = \inf_{x\in\mathbb{R}^N}\ G(x)$

G is coervive so admits a minimum. G is striclty convex as the sum of two striclty convex functions. Thus, the minimum is unique.

We consider a forward-backward algorithm.

In [ ]:
beta_grid = np.exp(np.log(10)*np.linspace(-4,0,5))

def proximal_gradient_norme1(lamb,K,y,T,display = False):
    tol = 1e-8
    m = K.shape[0]
    n = K.shape[1]
    w,v = np.linalg.eig(np.dot(K.T,K))
    epsilon = 1/np.real(np.max(w))
    u = np.zeros((n,1))
    norm2 = np.linalg.norm(np.dot(K,u) - y.reshape(m,1))
    norm1 = np.linalg.norm(u,1)
    obj = 0.5*norm2**2 + lamb * norm1
    last_obj = 0
    if display:
        print("Objectif : {}, ||Ku-y||^2 : {}, ||u|| : {}".format(obj,norm2**2,norm1))
    
    while abs(obj - last_obj) > tol:
        w = u - epsilon * np.dot(K.T,(np.dot(K,u) - y.reshape(m,1)))
        last_obj = obj
        u = prox_norme1(w,lamb,epsilon) 
        norm2 = np.linalg.norm(np.dot(K,u) - y.reshape(m,1))
        norm1 = np.linalg.norm(u,1)
        obj = 0.5*norm2**2 + lamb * norm1
        if display:
            print("Objectif : {}, ||Ku-y||^2 : {}, ||u|| : {}".format(obj,norm2**2,norm1)) 
    
    return u
        
def prox_norme1(w,lamb,epsilon):
    n = w.shape[0]
    out = np.zeros((n,1))
    for index in range(n):
        value = w[index,0]
        if value >  epsilon * lamb :
            out[index,0] = value - epsilon * lamb
        elif value <  -epsilon * lamb :
            out[index,0] = value + epsilon * lamb 
        elif value >=  -epsilon * lamb and value <= epsilon * lamb:
            out[index,0] = 0
    return out

for i in range(beta_grid.shape[0]):
    beta = beta_grid[i]
    x_hat = proximal_gradient_norme1(beta,K,y,T)
    print("MSE : {}, beta : {}".format(normalized_MSE(x_hat,x.reshape(200,1)),beta))
    plt.plot(np.log(T),x_hat)
    plt.plot(np.log(T),x)
    plt.grid()
    plt.show()

### Maximum entropy problem

We consider the problem:

$\inf_{x\in\mathbb{R}^N}\ \frac{1}{2}\ \lVert Kx - y \rVert^2_2 + \beta\ \mathrm{ent}(x) = \inf_{x\in\mathbb{R}^N}\ H(x)$

The entropy is strictly convex, and proper because its domain is $\mathbb{R}_+^N\neq\emptyset$. It is not differentiable because of the infinite values taken for negative values of the coordinates. The entropy is coercive  and the function norm is positive so $H$ admits a minimum. It is unique by strict convexity.

We first use a forward-backward algorithm.

$$\mathrm{prox}_{\varepsilon\beta\ \mathrm{ent}} (u) = \left(\varepsilon\beta\ W\left(\exp\left(\frac{u_i}{\varepsilon\beta} - 1 - \log(\varepsilon\beta)\right)\right)\right)_i$$ with the Lambert function W.

In [ ]:
# This function returns W(exp(u)) componentwise
def W_exp(u):
    
    m = u.shape[0]
    output = np.zeros((m,1))
    
    for index in range(m):
        val = u[index,0]
        
        if val >= 100:
            output[index,0] = val - np.log(val)
    
        if val < 100 and val > - 20 :
            w = 1
            v = 0
            while abs(w - v)/abs(w) > 1e-6:
                v = w
                e = np.exp(w)
                f = w*e - np.exp(val)
                w = w - f/((e*(w+1) - (w+2)*f/(2*w+2)))
            output[index,0] = w
            
        if val <=-20:
            output[index,0] = 0        
        
    return output

In [ ]:
beta_grid = np.exp(np.log(10)*np.linspace(-6,0,7))

def proximal_gradient_entr(lamb,K,y,T,xmin,xmax,display = False):
    tol = 1e-8
    m = K.shape[0]
    n = K.shape[1]
    u = np.zeros((n,1))
    w,v = np.linalg.eig(np.dot(K.T,K))
    epsilon = 1/np.real(np.max(w))
    norm2 = np.linalg.norm(np.dot(K,u) - y.reshape(m,1))
    entropy = ent(u)
    obj = 0.5 * norm2 **2 + lamb * entropy
    last_obj = 0
    if display:
        print("Objectif : {}, ||Ku-y||^2 : {}, entropy : {}".format(obj,norm2**2,entropy))
    
    while abs(obj - last_obj) > tol :
        w = u - epsilon * np.dot(K.T,np.dot(K,u) - y.reshape(m,1)) 
        last_obj = obj
        u = prox_entr(w/(lamb * epsilon) - (1+np.log(lamb*epsilon))*np.ones(w.shape),lamb,epsilon) 
        norm2 = np.linalg.norm(np.dot(K,u) - y.reshape(m,1))
        entropy = ent(u)
        obj = 0.5 * norm2 **2 + lamb * entropy
        if display:
            print("Objectif : {}, ||Ku-y||^2 : {}, entropy : {}".format(obj,norm2**2,entropy))
        
    return u
        
def prox_entr(w,lamb,epsilon):    
    return lamb*epsilon*W_exp(w)

def ent(x):
    tol = 1e-5
    n = x.shape[0]
    out = np.zeros((x.shape[0],1))
    for i in range(n):
        value = x[i]
        if value > tol:
            out[i] = value*np.log(value)
        else:
            out[i] = 0
    return sum(out)[0]
        

for i in range(beta_grid.shape[0]):
    beta = beta_grid[i]
    x_hat = proximal_gradient_entr(beta,K,y,T,xmin,xmax) #,True
    print("MSE : {}, beta : {}\n".format(normalized_MSE(x_hat,x.reshape(200,1)),beta))
    plt.plot(np.log(T),x_hat)
    plt.plot(np.log(T),x)
    plt.grid()
    plt.show()

We then use a Douglas-Rachford algorithm.

**Douglas Rashford Algorithm**

We define $f(x) = \frac{1}{2}\left\|Kx-y\right\|^{2} $ and $g(x) = \beta .\text{ent}(x) = \beta.\sum_{i=1}^{N}\phi(x^{(n)})$. To implement the Rashford-Douglas algorithm we need to compute $prox_{\gamma f}$ and $prox_{\gamma g}$:

$$prox_{\gamma f}(x) = (Id + \gamma K^{*}K)^{-1}(x+K^{*}y)$$

$$prox_{\gamma g}(x)  = \left( prox_{\gamma \beta \phi}\left (x^{(n)}\right)\right)_{n=1,..,N}$$

Where $prox_{\gamma \beta \phi}\left( x\right) =\beta \gamma W(e^{\frac{x}{\beta \gamma}-1 -ln(\beta \gamma)})$µ

In [ ]:
def prox_gamma_f(x, K, y, gamma = 1):
    n = x.shape[0]
    return np.linalg.solve(np.eye(n) + gamma*np.dot(K.T,K), x + np.dot(K.T,y).reshape(-1,1))

def prox_gamma_ent(x, gamma = 1):
    return gamma*W_exp((x/gamma)-1-np.log(gamma))
    

# We choose lambda = 1/it
def douglas_rashford(beta, K, y, x_init, gamma = 1, eps = 1e-3 , Iter_lim = 1000):
    it = 0
    x = x_init
    n = x.shape[0]
    matrix_temp = np.linalg.inv(np.eye(n) + gamma*np.dot(K.T,K))
    while (it < Iter_lim):
        it += 1
        y_ = prox_gamma_ent(x, gamma=gamma*beta)
        z = prox_gamma_f(2*y_-x, K=K, y=y, gamma=gamma)
        x = x + (1/it)*(z-y_)
    
    return z

In [ ]:
beta_grid = np.exp(np.log(10)*np.linspace(-2,2,7))

for i in range(beta_grid.shape[0]):
    beta = beta_grid[i]
    x_init = np.ones((K.shape[1],1))
    x_hat = douglas_rashford(beta = beta,K=K,y=y,x_init = x_init) #,True
    print("MSE : {}, beta : {}\n".format(normalized_MSE(x_hat,x.reshape(200,1)),beta))
    plt.plot(np.log(T),x_hat)
    plt.plot(np.log(T),x)
    plt.grid()
    plt.show()
        

$\underline{\textbf{Q8.}}$  We solve now the optimization problem $\underset{x \in \mathbb{R}^{N}}{\text{minimize}} \hspace{0.1cm}\text{ent(x)}$ subject to $\left\|Kx - y\right\|^{2} \leq \eta M\sigma^{2}$. This problem is equivalent to $$\underset{x \in \mathbb{R}^{N}}{\text{minimize}} \hspace{0.1cm} ent(x) + \iota_{C}(Kx) \hspace{1.5cm} (4)$$
Where $C = \left\{ u \in \mathbb{R}^{M} / \left\|u -y\right\|^{2}\leq \eta M \sigma^{2}\right\} = B(y,\sigma\sqrt{M\eta})$ 

Let's denote $g_{1}(x) = \text{ent}(x), \forall x \in \mathbb{R}^{N}$ and $g_{2}(u) = \iota_{C}(u), \forall u \in \mathbb{R}^{M} $. We denote also $L_{1} = Id_{N}$ and $L_{2} = K$. Thus the problem (4) can be written $$\underset{x \in \mathbb{R}^{N}}{\text{minimize}} \hspace{0.1cm} g_{1}(L_{1}x) + g_{2}(L_{2}x)$$

Thus we can use PPXA to solve this optimization problem, to do so we nedd to compute the proximity operator for $\gamma g_{1}$ and $\gamma g_{2}$: 

$$\forall x \in \mathbb{R}^{N}; prox_{\gamma g_{1}}(x)  = \left( prox_{\gamma  \phi}\left (x^{(n)}\right)\right)_{n=1,..,N}$$
$$\forall u \in \mathbb{R}^{M}; prox_{\gamma g_{2}}(u) = prox_{\iota_{C}}(u) = Proj_{B(y,\sigma\sqrt{M\eta})}(u) = y + (u-y).\text{max}(1,\frac{r}{\left\|u-y\right\|}) $$

In [ ]:
# Define of the projection
def proj_boule(u, y, eta = 1):
    return y + (u-y)*max(1,(sigma*np.sqrt(M*eta))/np.linalg.norm(u-y))
    
def ppxa(K, y, x_init, gamma = 1, eps = 1e-3 , Iter_lim = 1000):
    x_ = x_init
    x_1 = x_init
    x_2 = np.dot(K,x_1)
    v_ = np.linalg.solve(np.dot(K.T,K)  + np.eye(N), x_1 + np.dot(K.T,x_2))
    it = 0
    while (it < Iter_lim):
        it += 1
        lambda_ = 1 -1/it
        
        y_1 = prox_gamma_ent(x_1, gamma=gamma)
        y_2 = proj_boule(x_2, y, eta = 1)
        
        c = np.linalg.solve(np.dot(K.T,K)  + np.eye(N), y_1 + np.dot(K.T,y_2))
        
        x_1 = x_1 + lambda_*(2*c-v_ - y_1)
        x_2 = x_2 + lambda_*(np.dot(K,2*c-v_)-y_2)
        
        v_= v_ + lambda_*(c-v_)
        
    return v_
                         

In [ ]:
x_init = np.ones((K.shape[1],1))
x_hat = ppxa(K=K,y=y,x_init = x_init) 
print("MSE : {}, beta : {}\n".format(normalized_MSE(x_hat,x.reshape(200,1))))
plt.plot(np.log(T),x_hat)
plt.plot(np.log(T),x)
plt.grid()
plt.show()